In [1]:
!pip install transformers datasets evaluate accelerate

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from transformers import create_optimizer
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
from transformers.keras_callbacks import KerasMetricCallback
from transformers import pipeline
import nltk
from nltk.corpus import stopwords
from transformers.keras_callbacks import PushToHubCallback

In [3]:
dataset = load_dataset("Er1111c/Malicious_code_classification")
dataset['train'][0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'text': '\nfunction Test-EquivalentLocation {\n    [CmdletBinding()]\n    [OutputType("bool")]\n    param(\n        [Parameter(Mandatory = $false)]\n        [string]$LocationA,\n\n        [Parameter(Mandatory = $false)]\n        [string]$LocationB\n    )\n\n    $LocationA = $LocationA.TrimEnd("\\/")\n    $LocationB = $LocationB.TrimEnd("\\/")\n    return $LocationA -eq $LocationB\n}',
 'label': 0}

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
texttokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

# Apply the remove_stopwords function to each text entry in the dataset
dataset = dataset.map(lambda example: {'text': remove_stopwords(example['text'])})

def preprocess_function(data):
    return texttokenizer(data["text"], truncation=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
tokenized_news_dataset = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=texttokenizer, return_tensors="tf")
accuracy = evaluate.load("accuracy")

Map:   0%|          | 0/633 [00:00<?, ? examples/s]

In [7]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [8]:
id2label = {0: "No harm", 1: "Harm"}
label2id = {"Harm": 0, "No harm": 1}

In [9]:
batch_size = 4
num_epochs = 5
batches_per_epoch = len(tokenized_news_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-4, num_warmup_steps=0, num_train_steps=total_train_steps)

In [10]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    "google/flan-t5-base", num_labels=2, id2label=id2label, label2id=label2id
)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_news_dataset["train"],
    shuffle=True,
    batch_size=1,
    collate_fn=data_collator
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_news_dataset["test"],
    shuffle=False,
    batch_size=1,
    collate_fn=data_collator
)

In [12]:
model.compile(optimizer=optimizer)
push_to_hub_callback = PushToHubCallback(
    output_dir="code_classification_model",
    tokenizer=texttokenizer,
)
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
callbacks = [metric_callback,push_to_hub_callback]
history1 = model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/code_classification_model is already a clone of https://huggingface.co/Poojithpoosa/code_classification_model. Make sure you pull the latest changes with `repo.git_pull()`.


Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2532/2532 [==============================] - 460s 146ms/step - loss: 0.0990 - val_loss: 0.1110 - accuracy: 0.9621
Epoch 2/3
2532/2532 [==============================] - 300s 119ms/step - loss: 0.0194 - val_loss: 0.0382 - accuracy: 0.9858
Epoch 3/3
2532/2532 [==============================] - 299s 118ms/step - loss: 0.0152 - val_loss: 0.0382 - accuracy: 0.9858


In [13]:
text = 'param($days_old) (get-aduser -filter {enabled -eq $true} -Properties passwordlastset,employeenumber,whencreated | ? {($_.passwordlastset -gt (Get-Date).AddDays(-$days_old)) -and ($_.employeenumber) -and ($_.whenCreated -lt (Get-Date).AddDays(-$days_old))}).Countek.'

In [15]:
classifier = pipeline("sentiment-analysis", model="lakkan22/code_classification_model")
classifier(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some layers from the model checkpoint at poojithpoosa/code_classification_model were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at poojithpoosa/code_classification_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[{'label': 'No harm', 'score': 0.9936299920082092}]

In [20]:
optimizer, schedule = create_optimizer(init_lr=2e-6, num_warmup_steps=0, num_train_steps=total_train_steps)

model = TFAutoModelForSequenceClassification.from_pretrained(
    "google/flan-t5-base", num_labels=2, id2label=id2label, label2id=label2id
)
push_to_hub_callback = PushToHubCallback(
    output_dir="code_classification_model",
    tokenizer=texttokenizer,
)
model.compile(optimizer=optimizer)
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
callbacks = [metric_callback,push_to_hub_callback]
history2 = model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)
classifier = pipeline("sentiment-analysis", model="lakkan22/code_classification_model")
classifier(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned

Epoch 1/3
2532/2532 [==============================] - 435s 145ms/step - loss: 0.6917 - val_loss: 0.6723 - accuracy: 0.6319
Epoch 2/3
2532/2532 [==============================] - 317s 125ms/step - loss: 0.6689 - val_loss: 0.6699 - accuracy: 0.6319
Epoch 3/3
2532/2532 [==============================] - 312s 123ms/step - loss: 0.6700 - val_loss: 0.6699 - accuracy: 0.6319


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tf_model.h5:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some layers from the model checkpoint at poojithpoosa/code_classification_model were not used when initializing TFBertForSequenceClassification: ['dropout_265']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at poojithpoosa/code_classification_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


[{'label': 'Harm', 'score': 0.5212698578834534}]

In [ ]:

optimizer, schedule = create_optimizer(init_lr=2e-8, num_warmup_steps=0, num_train_steps=total_train_steps)

model = TFAutoModelForSequenceClassification.from_pretrained(
    "google/flan-t5-base", num_labels=2, id2label=id2label, label2id=label2id
)
push_to_hub_callback = PushToHubCallback(
    output_dir="code_classification_model",
    tokenizer=texttokenizer,
)
model.compile(optimizer=optimizer)
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)
callbacks = [metric_callback,push_to_hub_callback]
history3 = model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)
classifier = pipeline("sentiment-analysis", model="lakkan22/code_classification_model")
classifier(text)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/code_classification_model is already a clone of https://huggingface.co/Poojithpoosa/code_classification_mod

Epoch 1/3
2532/2532 [==============================] - ETA: 0s - loss: 0.6716